# Rank Scraper

Ce programme permet d'extraire les données d'un site web qui a publié un classement musical. Les données alimentent la base sqlite database.db dans le sous répertoire data.

### TODO
- Gothic
  - Kerrang! : https://www.kerrang.com/the-16-essential-goth-albums-you-need-to-know
    - no rank
- Indie Rock
  - https://www.melophobemusic.com/post/top-100-greatest-indie-rock-albums-of-all-time

In [271]:
pip install pandas



[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [272]:
params = { 
    "efe220cb-4981-40c7-8a6b-bd89b3569a17":  { 
        #10. Juju – Siouxsie and The Banshees
        "urls" : [
            "https://faroutmagazine.co.uk/the-10-greatest-goth-albums-of-all-time/"
        ],
        "regex" : r'(\d+)[.,]\s(.+?)\s?[–-]\s(.*?)(?:\s\(.*\))?$',
        "soup" : 'h3',
        "artist_first" : False,
        "name" : "Far Out",
        "title" : "The 10 greatest goth albums of all time",
        "style" : "Gothic",
        "year" : "",
        "date" : "22/05/2023",
        "language" : "en"
    },
    "b2065bc7-68e3-4c64-878f-3700735126dd":  { 
        #13. Red Lorry Yellow Lorry – Talk About the Weather (1985, Red Rhino)
        "urls" : [
            "https://www.brooklynvegan.com/classic-goths-13-greatest-albums/"
        ],
        "regex" : r'(\d+)[.,]\s(.+?)\s?[–-]\s(.*?)(?:\s\(.*\))?$',
        "soup" : 'strong',
        "artist_first" : True,
        "name" : "Brooklyn Vegan",
        "title" : "Classic Goth's 13 Greatest Albums",
        "style" : "Gothic",
        "year" : "",
        "date" : "30/10/2020",
        "language" : "en"
    },
    "116cbcd5-8721-49cb-8d1b-3cb897d8f338":  { 
        #1. Joy Division- Closer (Factory, 1980) 
        "urls" : [
            "https://post-punk.com/40-years-of-goth-essential-albums-from-the-genres-beginnings/"
        ],
        "regex" : r'(\d+)[.,]\s(.+?)\s?[–-]\s(.*?)(?:\s\(.*\))?$',
        "soup" : 'strong',
        "artist_first" : True,
        "name" : "Post-punk.com",
        "title" : "40 Years of Goth: Essential Albums from the Subculture’s Beginnings",
        "style" : "Gothic",
        "year" : "",
        "date" : "31/10/2017",
        "language" : "en"
    },
    "23d66e91-6dd9-4e1f-97db-984526ccc449":  { 
        #10. Franz Ferdinand de Franz Ferdinand (2004) 
        "urls" : [
            "https://www.gqmagazine.fr/article/meilleurs-albums-indie-rock"
        ],
        "regex" : r'(\d+)\.\s(.*?)\s(?:de|des)\s(.*?)\s\((\d+)\)',
        "soup" : 'h2',
        "artist_first" : False,
        "name" : "GQ Magazine",
        "title" : "Les 10 meilleurs albums d'indie rock de tous les temps",
        "style" : "Indie Rock",
        "year" : "",
        "date" : "02/10/2023",
        "language" : "fr"
    },
    "598ec7bb-043e-4fa6-b1e4-8e654571c961":  {  
        "urls" : [
            "https://www.nme.com/photos/the-500-greatest-albums-of-all-time-100-1-1426116"
        ],
        "regex" :  r"^(\d+)([^,'']+)[,']?\s*(.*?)'?$",
        "soup" : 'h2',
        "artist_first" : True,
        "name" : "NME",
        "title" : "The 500 Greatest Albums Of All Time",
        "style" : "",
        "year" : "",
        "date" : "25/10/2013",
        "language" : "en"
    },
    "598ec7bb-043e-4fa6-b1e4-8e654571c961":  {  
        "urls" : [
            "https://www.nme.com/photos/the-500-greatest-albums-of-all-time-200-101-1426258"
        ],
        "regex" :  r"^(\d+)([^,'']+)[,']?\s*(.*?)'?$",
        "soup" : 'h2',
        "artist_first" : True,
        "name" : "NME",
        "title" : "The 500 Greatest Albums Of All Time",
        "style" : "",
        "year" : "",
        "date" : "25/10/2013",
        "language" : "en"
    }
}

In [273]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import sqlite3
from datetime import datetime
import uuid
import html
import time
import json

In [274]:
def is_title(string):
    if string[0].isdigit():
        return True
    return False

In [275]:
# Liste d'URLs à traiter
url_list = ['https://spectrumculture.com/2013/03/21/13-best-goth-albums-of-all-time/',
'https://spectrumculture.com/2013/03/21/13-best-goth-albums-of-all-time/2', 
        'https://spectrumculture.com/2013/03/21/13-best-goth-albums-of-all-time/3']

In [276]:
def fetch_url(url):
    # Create a session
    session = requests.Session()

    # Change the user agent and add Referer header
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3',
        'Referer': 'http://www.google.com/'
    }

    response = session.get(url, headers=headers)
    if response.status_code == 200:
        return response.text
    else:
        print(f"Erreur lors de la récupération de la page. Code d'erreur : {response.status_code}")
        return None

    # Delay between requests
    time.sleep(1)

In [277]:
def parse_html(html, soup_string):
    soup = BeautifulSoup(html, 'html.parser')
    bold_titles = soup.find_all(soup_string)
    return bold_titles

In [278]:
def extract_data_from_title(title, regex_string, artist_first):
    title_string = title.get_text()
    title_string = re.sub(r'^\W+', '', title_string)  # Remove special characters and spaces from the left
    title_string = html.unescape(title_string)  # Convert HTML entities to normal characters
    print(title_string)
    if is_title(title_string):
        match = re.match(regex_string, title_string)
        if match:
            rank = match.group(1)
            if (artist_first):
                artist = match.group(2)
                title = match.group(3)
            else :
                title = match.group(2)
                artist = match.group(3)
            return {'rank': rank, 'artist': artist, 'title': title}
        else:
            print(f"Failed to extract data from title: {title_string}")
            return None
    else:
        return None

In [279]:
def create_dataframe(data_list, id):
    df = pd.DataFrame(data_list)
    df['id_ranking'] = id
    return df

In [280]:
def extract_spectrum(url_list, soup, string_regex, artist_first):
    data = []
    for url in url_list:
        html = fetch_url(url)
        if html:
            bold_titles = parse_html(html, soup)
            for title in bold_titles:
                data_item = extract_data_from_title(title, string_regex, artist_first)
                if data_item:
                    data.append(data_item)
    return data


In [281]:
# Create a connection to the SQLite database
# Doesn't matter if the database doesn't yet exist
def save_to_sqlite(df):
    conn = sqlite3.connect('./data/database.db')
    df.to_sql('album', conn, if_exists='append', index=False)
    conn.close()


In [282]:
def add_ranking(name, description, url, style, year, date, language):
    # Create a connection to the SQLite database
    conn = sqlite3.connect('./data/database.db')
    cursor = conn.cursor()

    # Check if ranking with same url already exists
    cursor.execute("SELECT id FROM ranking WHERE url = ?", (url,))
    data = cursor.fetchone()
    if data:
        # ranking already exists
        conn.close()
        return -1
    
    # Generate a unique ID
    guid = str(uuid.uuid4())
    
    # Define the insert query
    query = """
    INSERT INTO ranking (id, name, description, url, style, year, date, language, created_date)
    VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
    """
    # Define the record to insert
    record = (guid, name, description, url, style, year, date, language, datetime.now())

    # Execute the query
    cursor.execute(query, record)

    # Commit the transaction and close the connection
    conn.commit()
    conn.close()

    return guid

In [283]:
def delete_ranking(url) :
    conn = sqlite3.connect('./data/database.db')
    cursor = conn.cursor()
    # Get the ranking ID 
    cursor.execute("SELECT id FROM ranking WHERE url = ?", (url,))
    data = cursor.fetchone()
    if data:
        ranking_id = data[0]
        # Delete the ranking
        cursor.execute("DELETE FROM ranking WHERE url = ?", (url,))
        # Delete the albums
        cursor.execute("DELETE FROM albums WHERE id_ranking = ?", (ranking_id,))
        conn.commit()
        conn.close()
        return 0
    else:
        return -1

In [284]:
def add_ranking_spectrum_gothic():
    data=extract_spectrum(url_list)
    id = add_ranking('Spectrum Culture', 'The 13 Best Goth Albums of All Time', 'https://spectrumculture.com/2013/03/21/13-best-goth-albums-of-all-time/', 'Gothic','')
    df = create_dataframe(data,id)
    save_to_sqlite(df)

In [285]:
def extract_data_from_url(url, balise_html) :
    data = []  
    html = fetch_url(url)
    if html:
        titles = parse_html(html, balise_html)
        for title in titles:
            data.append(title.get_text())
    return data

In [286]:
def clean_data(data):
    data = [re.sub(r'^\W+', '', title) for title in data]
    data = [html.unescape(title) for title in data]
    #data = [title.replace(',', '') for title in data]
    #data = [title.replace("‘", ',') for title in data]    
    return data

In [287]:
def extract_data(data):
    extracted_data = []
    for title in data:
        match = re.match(r'(\d+)\.\s(.*?)\s–\s(.*?)\s\(.*\)$', title)
        if match:
            rank = match.group(1)
            artist = match.group(2)
            title = match.group(3)
            extracted_data.append({'rank': rank, 'artist': artist, 'title': title})
    return extracted_data


## Programme principal

In [288]:
# json_params = json.dumps(params)
# keys = list(params.keys())
# for key in keys[4:4]:    
#     urls = params[key]["urls"]
#     regex = params[key]["regex"]
#     artist_first = params[key]["artist_first"]
#     soup = params[key]["soup"]
#     data = extract_spectrum(urls, soup, regex, artist_first)
#     df = create_dataframe(data,'uuid')
#     ret = add_ranking(key, params[key]["name"], params[key]["title"], urls[0], params[key]["style"], params[key]["year"],params[key]["date"],params[key]["language"])
#     df['id_ranking'] = key
#     save_to_sqlite(df)
#     print(df)

# 0. Supprimer le classement
#delete_ranking('https://www.nme.com/photos/the-500-greatest-albums-of-all-time-100-1-1426116')
# 1. Récupérer les informations brutes depuis le site web
#data = extract_data_from_url("https://www.nme.com/photos/the-500-greatest-albums-of-all-time-100-1-1426116", "h2")
#data = extract_data_from_url("https://www.nme.com/photos/the-500-greatest-albums-of-all-time-200-101-1426258", "h2")
#data = extract_data_from_url("https://www.nme.com/photos/the-500-greatest-albums-of-all-time-300-201-1426482", "h2")
#data = extract_data_from_url("https://www.nme.com/photos/the-500-greatest-albums-of-all-time-400-301-1426436", "h2")
#data = extract_data_from_url("https://www.nme.com/photos/the-500-greatest-albums-of-all-time-500-401-1426363", "h2")
#data = extract_data_from_url("https://consequence.net/2022/09/best-albums-all-time-list/22/", "h2")
data = extract_data_from_url("https://www.magicrpm.com/les-100-meilleurs-albums-de-2023-n50-a-n1/", "p")
print(data)
# 2. Nettoyer les données
data_clean = clean_data(data)
print(data_clean)
# 3. Extraire les données de la liste (rank, artist, title)
data_extract = extract_data(data_clean)
print(data_extract)
# 4. Contrôle la position des données dans la liste
if (len(data_extract) == 50) :
    # 4. Créer le classement dans la base de données
    id = add_ranking('Magic', 'Les 101 meilleurs albums de 2023', 'https://www.magicrpm.com/les-100-meilleurs-albums-de-2023-n50-a-n1/', '', '2023', '29/12/2023', 'fr')
    df = create_dataframe(data_extract,id)
    #df = create_dataframe(data_extract,"5cced476-cbcb-4829-a110-1bd03ec1fb0c")
    # Convert rank to integer
    #df['rank'] = df['rank'].astype(int)
    # invert ranking and add 100
    #df['rank'] = 101 - df['rank'].astype(int)+400
    save_to_sqlite(df)
    print(data_extract)
else : 
    print("Erreur dans la récupération des données")



["Bienvenue dans la moitié haute de notre Top 101 de 2023 : les 50 premières places. Un classement à retrouver, en version longue et documentée, dans les 184 pages de notre hors-série bilan de l'année.", 'Retrouvez, dans les 184 pages de notre hors-série 2023, les 101 (!) disques de notre Top albums. Ils sont tous chroniqués et, pour une partie d’entre eux, enrichis de témoignages ou d’interview exclusives. Pour vous le procurer c’est par ici.', 'La première partie du classement (51e à 100e) est disponible ici.', '50. ALBIN DE LA SIMONE – Les Cent Prochaines Années (TÔT OU TARD)', '49. LE CRI DU CAIRE – Le Cri du Caire (LES DISQUES DU FESTIVAL PERMANENT / AIRFONO / L’ONDE & CYBÈLE)', '48. LAURA VEIRS – Phone Orphans (RAVEN MARCHING BAND RECORDS)', '47. ANNA B SAVAGE – in|FLUX (CITY SLANG)', '46. RÓISÍN MURPHY & DJ KOZE – Hit Parade (NINJA TUNE)', '45. HELENA DELAND – Goodnight Summerland (CHIVI CHIVI)', '44. GORILLAZ – Cracker Island (PARLOPHONE RECORDS / WARNER MUSIC)', '43. BOYGENIUS